## GigaAM from GitHub

### Installing reqs and testing

In [ ]:
! git clone https://github.com/salute-developers/GigaAM.git
%cd GigaAM

Cloning into 'gigaam'...
remote: Enumerating objects: 504, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 504 (delta 3), reused 7 (delta 2), pack-reused 492 (from 1)
Receiving objects: 100% (504/504), 2.75 MiB | 5.95 MiB/s, done.
Resolving deltas: 100% (298/298), done.
/content/gigaam


In [ ]:
! pip install -e .[tests]

In [ ]:
! pytest -v tests/test_loading.py -m partial --disable-warnings

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/gigaam
plugins: cov-7.0.0, hydra-core-1.3.2, anyio-4.11.0, typeguard-4.4.4, langsmith-0.4.42
collected 16 items / 12 deselected / 4 selected                                

tests/test_loading.py::test_model_revision_partial[emo] PASSED           [ 25%]
tests/test_loading.py::test_model_revision_partial[v2_ssl] PASSED        [ 50%]
tests/test_loading.py::test_model_revision_partial[v3_ctc] PASSED        [ 75%]
tests/test_loading.py::test_model_revision_partial[v3_e2e_rnnt] PASSED   [100%]

=========== 4 passed, 12 deselected, 2 warnings in 167.17s (0:02:47) ===========


### Model inference

##### Loading arguments

In [ ]:
import gigaam

model = gigaam.load_model(
    "v3_e2e_rnnt",      # model version: see readme#gigaam-family at GitHub for description
    fp16_encoder=True,  # enabled by default on CUDA, **IGNORED ON CPU (using fp32)**
    use_flash=False,    # disabled by default, boost performance for large tensors
    device=None,        # use cuda if available by default, can be set to `cpu` / `cuda`
)

##### Inference methods

In [ ]:
# Load test audio
audio_path = gigaam.utils.download_short_audio()

# Audio embeddings
model_name = "v2_ssl"       # Options: `v1_ssl`, `v2_ssl`, `v3_ssl`
model = gigaam.load_model(model_name)
embedding, _ = model.embed_audio(audio_path)
print("Embeds:", embedding)

# ASR
model_name = "v3_e2e_rnnt"  # Options: any model version with suffix `_ctc` or `_rnnt`
model = gigaam.load_model(model_name)
transcription = model.transcribe(audio_path)
print("\nTranscription:", transcription)

# Emotion recognition
model = gigaam.load_model("emo")
emotion2prob = model.get_probs(audio_path)
print("\nEmotions:", ", ".join([f"{emotion}: {prob:.3f}" for emotion, prob in emotion2prob.items()]))

Embeds: tensor([[[-0.2824,  0.3641,  0.4503,  ..., -0.4728, -0.4027, -0.2415],
         [ 0.1607, -0.4995, -0.0565,  ..., -0.6242, -0.2318, -0.2053],
         [-1.1857, -1.0032, -0.6091,  ..., -0.5142, -0.3736, -0.2652],
         ...,
         [ 0.0187, -0.3757, -0.8965,  ...,  0.1718,  0.0567,  0.1307],
         [ 0.2691, -0.0672, -0.5010,  ..., -1.4433, -1.4832, -1.4515],
         [-1.5648, -1.6692, -1.2828,  ...,  0.5110,  0.4826,  0.0133]]],
       device='cuda:0', grad_fn=<TransposeBackward0>)

Transcription: Ничьих не требуя похвал, Счастлив уж я надеждой сладкой, Что дева с трепетом любви Посмотрит, может быть, украдкой На песни грешные мои. У лукоморья дуб зелёный.

Emotions: angry: 0.000, sad: 0.002, neutral: 0.923, positive: 0.075


### ONNX convertation and inference

In [ ]:
onnx_dir = "onnx"

gigaam.load_model("v2_ssl").to_onnx(dir_path=onnx_dir)
gigaam.load_model("v3_e2e_rnnt").to_onnx(dir_path=onnx_dir)
gigaam.load_model("emo").to_onnx(dir_path=onnx_dir)

Succesfully ported onnx v2_ssl_encoder to onnx/v2_ssl_encoder.onnx.
Succesfully ported onnx v3_e2e_rnnt_encoder to onnx/v3_e2e_rnnt_encoder.onnx.
Succesfully ported onnx v3_e2e_rnnt_decoder to onnx/v3_e2e_rnnt_decoder.onnx.
Succesfully ported onnx v3_e2e_rnnt_joint to onnx/v3_e2e_rnnt_joint.onnx.
Succesfully ported onnx emo to onnx/emo.onnx.


In [ ]:
from gigaam.onnx_utils import load_onnx, infer_onnx

sessions, model_cfg = load_onnx(onnx_dir, "v2_ssl")
print("Embeds:", infer_onnx(audio_path, model_cfg, sessions))

sessions, model_cfg = load_onnx(onnx_dir, "v3_e2e_rnnt")
print("\nTranscription:", infer_onnx(audio_path, model_cfg, sessions))

sessions, model_cfg = load_onnx(onnx_dir, "emo")
print("\nEmotions:", infer_onnx(audio_path, model_cfg, sessions))

Embeds: [[[-0.2815268   0.36459672  0.45034242 ... -0.47318697 -0.40278506
   -0.24017537]
  [ 0.16104491 -0.49883014 -0.05703727 ... -0.6232359  -0.23104468
   -0.20491666]
  [-1.1852155  -1.002929   -0.608784   ... -0.5141611  -0.37379324
   -0.2654637 ]
  ...
  [ 0.01822925 -0.3753832  -0.8963473  ...  0.17242727  0.05693132
    0.1306364 ]
  [ 0.26943898 -0.06628752 -0.5013158  ... -1.4431518  -1.4824082
   -1.4502244 ]
  [-1.5649999  -1.669118   -1.2826786  ...  0.511411    0.48322338
    0.01353467]]]

Transcription: Ничьих не требуя похвал, Счастлив уж я надеждой сладкой, Что дева с трепетом любви Посмотрит, может быть, украдкой На песни грешные мои. У лукоморья дуб зелёный.

Emotions: [[7.7093333e-05 2.2028047e-03 9.2327267e-01 7.4447356e-02]]


### Longform

As `.transcribe` function input lenght is limited by 25 seconds, we need `.transcribe_longform` with audio segmentation based on `pyannote/segmentation-3.0` voice activity detection pipeline.

In [ ]:
! pip install -e .[longform]

In [ ]:
# in colab the session might need restarting here
%cd GigaAM

/content/gigaam


In [ ]:
! HF_TOKEN="<your hf token>" pytest -v tests/test_longform.py --disable-warnings

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/gigaam
plugins: cov-7.0.0, hydra-core-1.3.2, anyio-4.11.0, typeguard-4.4.4, langsmith-0.4.42
collected 7 items                                                              

tests/test_longform.py::test_segmentation_functionality[30.0] PASSED     [ 14%]
tests/test_longform.py::test_segmentation_functionality[60.0] PASSED     [ 28%]
tests/test_longform.py::test_segmentation_functionality[120.0] PASSED    [ 42%]
tests/test_longform.py::test_transcribe_longform[v3_ctc] PASSED          [ 57%]
tests/test_longform.py::test_transcribe_longform[v3_e2e_rnnt] PASSED     [ 71%]
tests/test_longform.py::test_longform_consistency[v3_ctc] PASSED         [ 85%]
tests/test_longform.py::test_segmentation_edge_cases PASSED              [100%]

======================== 7 passed, 1 warning in 56.39s ======

In [ ]:
import os
import warnings
warnings.simplefilter("ignore")

import gigaam

os.environ["HF_TOKEN"] = "<HF_TOKEN with read access to `pyannote/segmentation-3.0`>"

long_audio_path = gigaam.utils.download_long_audio()
model = gigaam.load_model("v3_e2e_rnnt")

utterances = model.transcribe_longform(long_audio_path)
for utt in utterances:
   transcription, (start, end) = utt["transcription"], utt["boundaries"]
   print(f"[{gigaam.format_time(start)} - {gigaam.format_time(end)}]: {transcription}")

[00:00:00 - 00:16:80]: Вечерня отошла давно, Но в кельях тихо и темно; Уже и сам игумен строгий Свои молитвы прекратил И кости ветхие склонил, Перекрестясь на одр убогий. Кругом и сон, и тишина; Но церкви дверь отворена.
[00:17:07 - 00:32:54]: Трепещет луч лампады, И тускло озаряет он И тёмную живопись икон, и возглащённые оклады. И раздаётся в тишине То тяжкий вздох, то шёпот важный, И мрачно дремлет в тишине старинный свод.
[00:32:95 - 00:49:30]: Глухой и влажный Стоят за клиросом чернец и грешник, Неподвижны оба. И шёпот их — Как глаз из гроба, И грешник бледен, как мертвец — Монах. Несчастный! Полно, перестань!
[00:49:81 - 01:05:65]: Ужасна исповедь злодея, Заплачена тобою дань Тому, Кто в злобе пламенея Лукавого грешника блюдёт И к вечной гибели ведёт. Смирись, опомнись. Время, время. Раскаянье, покров
[01:05:94 - 01:10:88]: Я разрешу тебя, грехов сложи мучительное бремя.


### More advanced examples

##### Another way to load audio

In [ ]:
import librosa
import torch

from gigaam.utils import AudioDataset
from gigaam.preprocess import SAMPLE_RATE as SR

fname = gigaam.utils.download_short_audio()
wav = gigaam.load_audio(fname)
# arrays can be not equal after possible resampling, but close enough
wav_ = torch.from_numpy(librosa.load(fname, sr=SR, mono=True)[0])

wav_tns, lengths = AudioDataset.collate([wav, wav_])
with torch.no_grad():
    encoded, encoded_len = model(
        wav_tns.to(model._device).to(model._dtype), lengths.to(model._device)
    )
    print(model.decoding.decode(model.head, encoded, encoded_len))

# outputs expected to be equal

['Ничьих не требуя похвал, Счастлив уж я надеждой сладкой, Что дева с трепетом любви Посмотрит, может быть, украдкой На песни грешные мои. У лукоморья дуб зелёный.', 'Ничьих не требуя похвал, Счастлив уж я надеждой сладкой, Что дева с трепетом любви Посмотрит, может быть, украдкой На песни грешные мои. У лукоморья дуб зелёный.']


##### Longform & batch > 1

We'll demonstrate batching with an example for longform inference. Here a long audio is split into independent segments - making it ideal for batching.

In [ ]:
import os
import soundfile as sf
from pathlib import Path

import gigaam
from gigaam.preprocess import SAMPLE_RATE as SR
from gigaam.vad_utils import segment_audio_file


os.environ["HF_TOKEN"] = "<HF_TOKEN with read access to `pyannote/segmentation-3.0`>"

exp_dir = Path("tmp_inference_example")
exp_dir.mkdir(exist_ok=True)
long_fname = gigaam.utils.download_long_audio()
model = gigaam.load_model("v3_e2e_rnnt")

# Saving parts as files
with torch.inference_mode():
    segments, boundaries = segment_audio_file(long_fname, sr=SR, device=model._device)
wav_paths = []
for i, segment in enumerate(segments):
    wav_paths.append(str(exp_dir / f"{Path(long_fname).stem}_{i}{Path(long_fname).suffix}"))
    sf.write(wav_paths[-1], segment, SR)

# Load wavs: you can use their paths or load with librosa-like method as above
wavs_np = [librosa.load(wav_path, sr=SR, mono=True)[0] for wav_path in wav_paths]

Finally, run batched inference. In our case the input audio is fairly short, `batch_size` can be decreased for longer inputs.

In [ ]:
import torch

dataset = AudioDataset(wavs_np)  # or AudioDataset(wav_paths)
batch_size = len(dataset)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, collate_fn=dataset.collate, shuffle=False
)

pred_texts = []
for wav_tns, lengths in dataloader:
    wav_tns, lengths = wav_tns.to(model._device).to(model._dtype), lengths.to(model._device)
    with torch.no_grad():
        encoded, encoded_len = model(wav_tns, lengths)
    pred_texts.extend(model.decoding.decode(model.head, encoded, encoded_len))

for (start, end), text in zip(boundaries, pred_texts):
    print(f"[{gigaam.format_time(start)} - {gigaam.format_time(end)}]: {text}")

[00:00:00 - 00:16:80]: Вечерня отошла давно, Но в кельях тихо и темно; Уже и сам игумен строгий Свои молитвы прекратил И кости ветхие склонил, Перекрестясь на одр убогий. Кругом и сон, и тишина; Но церкви дверь отворена.
[00:17:07 - 00:32:54]: Трепещет луч лампады, И тускло озаряет он И тёмную живопись икон, и возглащённые оклады. И раздаётся в тишине То тяжкий вздох, то шёпот важный, И мрачно дремлет в тишине старинный свод.
[00:32:95 - 00:49:30]: Глухой и влажный Стоят за клиросом чернец и грешник, Неподвижны оба. И шёпот их — Как глаз из гроба, И грешник бледен, как мертвец — Монах. Несчастный! Полно, перестань!
[00:49:81 - 01:05:65]: Ужасна исповедь злодея, Заплачена тобою дань Тому, Кто в злобе пламенея Лукавого грешника блюдёт И к вечной гибели ведёт. Смирись, опомнись. Время, время. Раскаянье, покров
[01:05:94 - 01:10:88]: Я разрешу тебя, грехов сложи мучительное бремя.


## GigaAM from Hugging Face

### Reqs and files loading: now by hand

In [ ]:
! pip install numpy==2.* torch==2.8.* torchaudio==2.8.* transformers==4.57.* \
    pyannote.audio==4.0 torchcodec==0.7 numba>=0.62 \
    onnx==1.19.* onnxruntime==1.23.* \
    hydra-core==1.3.* omegaconf==2.3.* \
    sentencepiece tqdm
! pip install --force-reinstall numpy  # make transformers work fine

In [ ]:
! wget https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/example.wav
! wget https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/long_example.wav

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("ai-sage/GigaAM-v3", revision="e2e_rnnt", trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

modeling_gigaam.py:   0%|          | 0.00/49.1k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/449M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/255k [00:00<?, ?B/s]

### Everything else is the same!

> Note: `fp16` encoder on CUDA can be used with `model.model.encoder = model.model.encoder.half()`.

In [ ]:
import torch

device = torch.device("cpu")
repo_name = "ai-sage/GigaAM-v3"

# Audio embeddings
model_name = "ssl"
model = AutoModel.from_pretrained(repo_name, revision=model_name, trust_remote_code=True).to(device)
embedding, _ = model.embed_audio("example.wav")
print("Embed:", embedding)

# ASR
model_name = "e2e_rnnt"  # Options: rnnt, ctc, e2e_rnnt, e2e_ctc
model = AutoModel.from_pretrained(repo_name, revision=model_name, trust_remote_code=True).to(device)
transcription = model.transcribe("example.wav")
print("\nTranscription:", transcription)

Embed: tensor([[[-0.2815,  0.3646,  0.4503,  ..., -0.4732, -0.4028, -0.2402],
         [ 0.1610, -0.4988, -0.0570,  ..., -0.6232, -0.2310, -0.2049],
         [-1.1852, -1.0029, -0.6088,  ..., -0.5142, -0.3738, -0.2655],
         ...,
         [ 0.0182, -0.3754, -0.8963,  ...,  0.1724,  0.0569,  0.1306],
         [ 0.2694, -0.0663, -0.5013,  ..., -1.4432, -1.4824, -1.4502],
         [-1.5650, -1.6691, -1.2827,  ...,  0.5114,  0.4832,  0.0135]]],
       grad_fn=<TransposeBackward0>)

Transcription: Ничьих не требуя похвал, Счастлив уж я надеждой сладкой, Что дева с трепетом любви Посмотрит, может быть, украдкой На песни грешные мои. У лукоморья дуб зелёный.


In [ ]:
import os
os.environ["HF_TOKEN"] = "<HF_TOKEN with read access to `pyannote/segmentation-3.0`>"

model = AutoModel.from_pretrained(repo_name, revision="e2e_rnnt", trust_remote_code=True)
utterances = model.transcribe_longform("long_example.wav")
print("Longform transcription:\n")
for utt in utterances:
   transcription, (start, end) = utt["transcription"], utt["boundaries"]
   print(f"[{start:.4f} - {end:.4f}]: {transcription}")

Longform transcription:

[0.0000 - 16.8047]: Вечерня отошла давно, Но в кельях тихо и темно; Уже и сам игумен строгий Свои молитвы прекратил И кости ветхие склонил, Перекрестясь на одр убогий. Кругом и сон, и тишина; Но церкви дверь отворена.
[17.0747 - 32.5491]: Трепещет луч лампады, И тускло озаряет он И тёмную живопись икон, и возглащённые оклады. И раздаётся в тишине То тяжкий вздох, то шёпот важный, И мрачно дремлет в тишине старинный свод.
[32.9541 - 49.3060]: Глухой и влажный Стоят за клиросом чернец и грешник, Неподвижны оба. И шёпот их — Как глаз из гроба, И грешник бледен, как мертвец — Монах. Несчастный! Полно, перестань!
[49.8122 - 65.6578]: Ужасна исповедь злодея, Заплачена тобою дань Тому, Кто в злобе пламенея Лукавого грешника блюдёт И к вечной гибели ведёт. Смирись, опомнись. Время, время. Раскаянье, покров
[65.9447 - 70.8891]: Я разрешу тебя, грехов сложи мучительное бремя.


In [ ]:
model.to_onnx("hf_onnx")

Succesfully ported onnx v3_e2e_rnnt_encoder to hf_onnx/v3_e2e_rnnt_encoder.onnx.
Succesfully ported onnx v3_e2e_rnnt_decoder to hf_onnx/v3_e2e_rnnt_decoder.onnx.
Succesfully ported onnx v3_e2e_rnnt_joint to hf_onnx/v3_e2e_rnnt_joint.onnx.
